<a href="https://colab.research.google.com/github/d9249/DACON/blob/main/Inference%20(Dataset%20statistics).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Oct  6 05:12:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !mkdir open_data
!unzip -uq "/content/drive/MyDrive/DACON_2021_recognize_traffic_signal_motion/new_open.zip" -d "/content/"

In [4]:
!cp /content/drive/MyDrive/pytorch_python/dataset.py .
!cp /content/drive/MyDrive/pytorch_python/specific_augmentation.py .
!cp /content/drive/MyDrive/pytorch_python/specific_dataset.py .
!cp /content/drive/MyDrive/pytorch_python/specific_train.py .
!cp /content/drive/MyDrive/pytorch_python/train.py .

In [5]:
import torch, torchvision
import os, json, PIL

import pandas as pd
import torchvision.transforms as transforms

from train import *

In [9]:
!cp /content/drive/MyDrive/DACON_2021_recognize_traffic_signal_motion/model/Dataset_statistics/5_40.pt .

In [10]:
model = Baseline(model='resnet18', num_classes=157)
model.model.load_state_dict(torch.load('./5_40.pt'))
model.model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
test_transform = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop((224, 400)),
    transforms.ToTensor(), transforms.Normalize(mean=[0.3005, 0.3035, 0.2712], std=[0.1519, 0.1252, 0.0996])])


data_dir = './test/'

predictions = []
with torch.no_grad():
    for num in sorted(os.listdir(data_dir)):
        with open(data_dir + '{}/{}.json'.format(num, num), 'r') as j:
            temp = json.load(j)
            imgs = []
            for info in temp['annotations']:
                img_dir = data_dir + '{}/{}.png'.format(num, info['image_id'])
                img = PIL.Image.open(img_dir).convert('RGB')
                img = test_transform(img)
                imgs.append(img)
            imgs = torch.stack(imgs).cuda()
            prediction = torch.nn.Softmax(dim=1)(model.model(imgs))
            prediction = torch.mean(prediction, dim=0)
            #if torch.max(prediction) > 0.9:
            #    prediction[torch.argmax(prediction)] = 1
            #    prediction[prediction < 1] = 0
            
            if torch.sum(prediction) != 1: print(torch.sum(prediction))
            predictions.append(prediction.cpu().numpy())

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(1

In [13]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission.iloc[:,1:] = predictions
display(sample_submission.head())
sample_submission.to_csv('./DatasetStat_5_40_threshno.csv', index=False)

,Image_Path,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7,Label_8,Label_9,Label_10,Label_11,Label_12,Label_13,Label_14,Label_15,Label_16,Label_17,Label_18,Label_19,Label_22,Label_23,Label_25,Label_26,Label_27,Label_28,Label_29,Label_31,Label_32,Label_34,Label_35,Label_36,Label_37,Label_39,Label_40,Label_41,Label_42,Label_43,Label_44,...,Label_142,Label_143,Label_144,Label_145,Label_146,Label_147,Label_148,Label_149,Label_150,Label_151,Label_153,Label_154,Label_155,Label_156,Label_157,Label_158,Label_159,Label_160,Label_161,Label_162,Label_163,Label_165,Label_167,Label_168,Label_169,Label_171,Label_172,Label_173,Label_174,Label_175,Label_177,Label_186,Label_188,Label_189,Label_190,Label_191,Label_192,Label_193,Label_194,Label_195
0,./test\649,0.367728,0.006667,0.001476,0.002667,0.000485,0.039772,0.031327,0.000344,0.000106,0.001023,0.003411,0.000285,0.001021,0.000140,0.000100,0.000535,0.000576,0.000401,0.001371,0.000918,0.002131,0.020983,0.013615,0.003787,0.014786,0.006742,0.000444,0.000223,0.001282,0.000242,0.318240,0.002871,0.000774,0.000711,0.000458,0.000722,0.005839,0.000804,0.002776,...,0.001246,0.000092,0.000080,0.000480,0.000047,0.000305,0.000098,0.000807,0.000283,0.000168,0.000073,0.000102,0.000034,0.000008,0.000197,0.000126,0.000102,0.001489,0.000025,0.000016,0.000068,0.000511,0.000445,0.000230,0.000158,0.000165,0.000129,0.000451,0.000140,0.000182,0.000481,0.000093,0.000077,0.000264,0.000087,0.000142,0.000167,0.000065,0.000096,0.000035
1,./test\650,0.001189,0.000086,0.000201,0.000151,0.000017,0.000017,0.000719,0.000187,0.000082,0.000034,0.836677,0.000293,0.000376,0.000183,0.000043,0.000345,0.021084,0.000069,0.000048,0.000049,0.000044,0.000577,0.000043,0.000037,0.000113,0.001215,0.000160,0.000046,0.000278,0.000043,0.000069,0.000536,0.000254,0.000240,0.000105,0.000113,0.063601,0.000114,0.000565,...,0.000029,0.000012,0.000024,0.000041,0.000052,0.000011,0.000607,0.000003,0.000010,0.000008,0.000106,0.000007,0.000013,0.000009,0.000009,0.000022,0.000007,0.000005,0.000035,0.000090,0.000010,0.000005,0.023384,0.000012,0.000329,0.000028,0.000030,0.000353,0.000102,0.000034,0.000132,0.000029,0.000025,0.000099,0.000114,0.000004,0.000050,0.000010,0.000030,0.000005
2,./test\651,0.000981,0.007770,0.016602,0.003632,0.000130,0.000094,0.000272,0.002707,0.001187,0.000481,0.198697,0.591387,0.009874,0.000888,0.000671,0.000536,0.004212,0.072880,0.000828,0.000722,0.000342,0.000356,0.000225,0.000279,0.000766,0.000656,0.000276,0.000222,0.000198,0.000186,0.000116,0.000261,0.000475,0.000656,0.003888,0.000284,0.004185,0.000342,0.000259,...,0.000256,0.000076,0.000160,0.000574,0.000146,0.000161,0.000087,0.000019,0.000130,0.000077,0.000236,0.000027,0.000270,0.000072,0.000056,0.000090,0.000033,0.000019,0.000083,0.000167,0.000083,0.000116,0.004819,0.000032,0.000197,0.000158,0.000168,0.000768,0.000072,0.000087,0.001138,0.000205,0.000077,0.000175,0.000247,0.000053,0.000185,0.000096,0.000602,0.000074
3,./test\652,0.000197,0.002676,0.001582,0.000758,0.000026,0.000028,0.000060,0.000771,0.000279,0.000183,0.018843,0.935780,0.001791,0.000314,0.000076,0.000085,0.000202,0.022718,0.000114,0.000218,0.000068,0.000071,0.000093,0.000097,0.000336,0.000154,0.000052,0.000040,0.000033,0.000037,0.000030,0.000026,0.000057,0.000096,0.000938,0.000062,0.000206,0.000047,0.000039,...,0.000039,0.000024,0.000035,0.000079,0.000018,0.000034,0.000010,0.000006,0.000031,0.000022,0.000023,0.000010,0.000068,0.000021,0.000024,0.000013,0.000009,0.000010,0.000018,0.000016,0.000033,0.000041,0.000230,0.000005,0.000024,0.000025,0.000027,0.000076,0.000009,0.000015,0.000146,0.000035,0.000008,0.000022,0.000037,0.000017,0.000038,0.000025,0.000090,0.000023
4,./test\653,0.000063,0.000191,0.006541,0.001512,0.000056,0.000017,0.000019,0.000060,0.000145,0.000110,0.000445,0.014743,0.935024,0.027956,0.000091,0.000038,0.000023,0.000063,0.001764,0.000430,0.000148,0.000231,0.000191,0.000100,0.000275,0.000085,0.000049,0.000013,0.000056,0.000043,0.000101,0.000016,0.000049,0.000225,0.000095,0.0